In [1]:
!pip install lightgbm -i https://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


### 直接使用lightgbm库

In [8]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import numpy as np
import pandas as pd

from sklearn.metrics import precision_score, recall_score

# 导入数据
iris = datasets.load_iris()
data = iris.data
label = iris.target
print(pd.DataFrame(data).head())
print(pd.DataFrame(label).head())

data1 = pd.DataFrame(data)
data1.columns = ['sepal_l','sepal_w','petal_l','petal_w']
print(data1.head())

label1 = pd.DataFrame(label)
label1.columns = ["label"]
print(label1.head())

     0    1    2    3
0  5.1  3.5  1.4  0.2
1  4.9  3.0  1.4  0.2
2  4.7  3.2  1.3  0.2
3  4.6  3.1  1.5  0.2
4  5.0  3.6  1.4  0.2
   0
0  0
1  0
2  0
3  0
4  0
   sepal_l  sepal_w  petal_l  petal_w
0      5.1      3.5      1.4      0.2
1      4.9      3.0      1.4      0.2
2      4.7      3.2      1.3      0.2
3      4.6      3.1      1.5      0.2
4      5.0      3.6      1.4      0.2
   label
0      0
1      0
2      0
3      0
4      0


In [9]:
# 划分数据
train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)
print("训练集长度:", len(train_x))
print("测试集长度:", len(test_x))

训练集长度: 105
测试集长度: 45


In [13]:
# 转换数据格式
train_data = lgb.Dataset(train_x, train_y)
test_data = lgb.Dataset(test_x, test_y)

# 设置参数
lgb_params = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "metric": "multi_error",
    "verbose": 1 , # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
    "num_class":3 #lightgbm.basic.LightGBMError: b'Number of classes should be specified and greater than 1 for multiclass training'

}

# 模型训练
clf = lgb.train(lgb_params, train_data,num_boost_round=10,
               valid_sets=[train_data, test_data],
               verbose_eval=10)

# 模型预测
test_pre = clf.predict(test_x, num_iteration=clf.best_iteration)
print(test_pre[:5])

# 选择最大概率的列
test_pre_1 = np.asarray([np.argmax(row) for row in test_pre])
print("test result:", test_pre_1)

# 模型评估
print('test precision:',precision_score(test_y, test_pre_1, average='macro')) 
print('test recall:',recall_score(test_y, test_pre_1, average='macro'))  

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000216 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 86
[LightGBM] [Info] Number of data points in the train set: 105, number of used features: 4
[LightGBM] [Info] Start training from score -1.219973
[LightGBM] [Info] Start training from score -1.043042
[LightGBM] [Info] Start training from score -1.043042
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

### Sklearn 接口形式使用lightgbm

In [15]:
import lightgbm as lgb
lgb_params = {
    "learning_rate":0.1,
    "max_bin":150,
    "num_leaves":32,
    "max_depth":11,
    "objective":"multiclass",
    "n_estimators":300
}

model = lgb.LGBMClassifier(**lgb_params)

model.fit(train_x, train_y)

# 预测
test_pre_2 = model.predict(test_x)
print(test_pre_2)
# 模型评估
print('test precision：',precision_score(test_y, test_pre_2, average='macro')) 
print('test recall：',recall_score(test_y, test_pre_2, average='macro'))  

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
test precision： 1.0
test recall： 1.0


F:\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
F:\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### 总结

1. lgb.train中正则化参数为"lambda_l1", "lambda_l1"，sklearn中则为'reg_alpha', 'reg_lambda'。
2. 多分类时lgb.train除了'objective':'multiclass',还要指定"num_class":5，而sklearn接口只需要指定'objective':'multiclass'。
3. 迭代次数在sklearn中是'n_estimators':20，在初始化模型时指定